# **Kegiatan Modul 4**

Dalam mendefinisikan model Machine Learing ada banyak cara untuk mendapatkan model dengan performa terbaik salah satunya adalah hyperparameter tuning. Contoh hyperparameter tuning telah diberikan dalam materi di modul ini, dalam kegiatan ini telah diberikan sedikit contoh lain untuk mendefinisikan model hyperparameter tuning dengan hparam. **Tugas anda dalam kegiatan modul kali ini adalah membuat model CNN dengan hyperparameter tuning dengan ketentuan wajib sebagai berikut :**
* Ketentuan wajib dataset minimal data 3000
* Lakukan eksperimen model untuk mencari model CNN dengan performa terbaik untuk mengklasifikasikan dataset yang sama dalam contoh dibawah.
* Definisikan minimal 5 model CNN dengan komposisi parameter yang berbeda-beda.
* Dalam mendefinisikan model, WAJIB melakukan tuning minimal pada 3 parameter model yang dirubah misalnya merubah jumlah neuron, nilai dropout, nilai lerning rate, jenis optimizer, jenis activation function dsb.
* Visualisasikan hasil evaluasi performa model. Anda dapat menggunakan plot pada umumnya seperti grafik loss dan accuracy dengan matplotib atau visualisasi interaktif menggunakan tensorboard seperti pada contoh dibawah.



### **NOTE :** 
Untuk mendapatkan penilaian syarat wajib diatas harus terpenuhi semua. Tunjukkan kepada asisten bahwa anda telah melakukan semua syarat wajib diatas serta jelaskan semampu anda.


**Poin penilaian :**

* Menyelesaikan syarat wajib diatas untuk mendapatkan minimal **50 poin**
* Dapat menunjukkan dan menjelaskan hasil evaluasi masing-masing model yang digunakan. **5 poin**
* Dapat memvisualisasikan hasil evaluasi model dan menjelaskannya kepada asisten. **5 poin**
* Menggunakan maptplotlib atau seaborn untuk visualisasi loss dan accuracy. **10 poin**
* Menggunakan tensorboard untuk visualisasi hasil evaluasi model menggunakan hyperparameter tuning. **15 poin**
* Menggunakan Hparam untuk tuning model. **10 poin**
* Hasil accuracy model terbaik > 70% **nilai tambahan 5 poin** *
* Hasil accuracy model terbaik >= 80% **nilai tambahan 10 poin** *
* Hasil accuracy model terbaik >= 85% **nilai tambahan 15 poin** *
* Hasil accuracy model terbaik >= 90% **nilai tambahan 25 poin** *
* Hasil loss model terbaik < 10% **10 poin**

**Ekstra 10 poin bagi anda yang dapat menjelaskan pekerjaan anda dengan baik dan tanpa plagiasi.**

\* *Pilih salah satu berdasarkan nilai akurasi terbaik*

## **Contoh penggunaan hparam dan tensorboard:**

### Download dan unzip dataset.

In [ ]:
# Download dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip 

--2020-07-31 10:46:22--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c08::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  67.2MB/s    in 1.0s    

2020-07-31 10:46:23 (67.2 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [ ]:
!unzip \cats_and_dogs_filtered.zip &> /dev/null 

### Masuk ke lokasi tempat menyimpan dataset

In [ ]:
%cd /content/drive/My Drive/RESEARCH CENTER/Kaggle

/content/drive/My Drive/RESEARCH CENTER/Kaggle


### Definisi data generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

training_set = train_datagen.flow_from_directory("cats_and_dogs_filtered/train/",
                                                 target_size=(128,128),
                                                 color_mode="rgb",
                                                 shuffle=False,
                                                 batch_size = 8,
                                                 class_mode='categorical')

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        rescale=1./255,
                                        shear_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')

validation_set = validation_datagen.flow_from_directory("cats_and_dogs_filtered/validation/",
                                                        target_size=(128,128),
                                                        color_mode="rgb",
                                                        shuffle=False,
                                                        batch_size = 8,
                                                        class_mode='categorical')

Found 1000 images belonging to 2 classes.


### Load TensorBoard notebook extension dan Hapus semua **logs** sebelumnya

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
!rm -rf ./logs/ 

### Definisikan hparam yang ingin digunakan. 

Semakin banyak hparam yang didefinisikan maka semakin banyak pula kombinasi hyperparameter digunakan untuk training model. Semakin banyak kombinasi maka semakin memakan banyak waktu (FYI : lama training model yang digunakan dalam contoh ini +/- 3-4 jam menggunakan GPU).

Menyimpan semua summary hparam kedalam `logs/hparam_tuning` untuk digunakan oleh tensorboard dan sebagai penyimpanan sementara training information.

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'adadelta']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

### Definisikan `x_val dan y_val` dari validation data untuk evaluasi model.

In [ ]:
xvl, yvl = zip(*(validation_set[i] for i in range(len(validation_set))))
x_val, y_val = np.vstack(xvl), np.vstack(yvl)

### Definisikan Model menggunakan hparam.

Disini anda mendefinisikan model, compile model, training model secara bersamaan didalam fungsi `train_test_model`. Fungsi ini akan mengembalikan nilai akurasi rata-rata dari masing-masing model.

In [ ]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(64, (5, 5), 
                                                             activation=tf.nn.relu, 
                                                             input_shape=(128,128,3)),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Conv2D(64, (5, 5), 
                                                             activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Conv2D(32, (5, 5), 
                                                             activation=tf.nn.relu),
                                      tf.keras.layers.BatchNormalization(),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Flatten(),
                                      tf.keras.layers.Dense(hparams[HP_NUM_UNITS], 
                                                            activation=tf.nn.relu),
                                      tf.keras.layers.Dense(64, activation=tf.nn.relu),
                                      tf.keras.layers.Dense(32, activation=tf.nn.relu),
                                      tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
                                      tf.keras.layers.Dense(2, activation=tf.nn.softmax),
                                      ])

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(training_set, 
            validation_data=validation_set, 
            epochs=25,
            )
  _, accuracy = model.evaluate(x_val, y_val)
  return accuracy

### Definisikan fungsi `run` 

Hal ini bertujuan untuk melakukan log record hparams yang berisi summary dari hyperparameters dan final accuracy-nya.

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=50)

### Grid Search Training Model


Mulai jalankan training dan catat semuanya di bawah satu direktori induk.
Sekarang Anda dapat mencoba beberapa percobaan, latih masing-masing dengan serangkaian parameter yang berbeda. Untuk kesederhanaan, gunakan **grid search**: coba semua kombinasi parameter diskrit dan hanya batas bawah dan atas dari parameter bernilai riil. Untuk skenario yang lebih kompleks, mungkin lebih efektif untuk memilih setiap nilai hyperparameter secara acak (ini disebut **random search**). Ada metode yang lebih maju yang dapat digunakan.

Jalankan beberapa percobaan, hal ini akan memakan waktu yang cukup lama:

In [ ]:
['adam', 'sgd', 'adadelta', 'rmsprop', 'adamx', 'nadam']

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 64, 'dropout': 0.2, 'optimizer': 'adadelta'}
Epoch 1/25
250/250 [==============================] - 22s 90ms/step - loss: 0.9160 - accuracy: 0.5055 - val_loss: 0.7463 - val_accuracy: 0.5000
Epoch 2/25
250/250 [==============================] - 22s 89ms/step - loss: 0.9109 - accuracy: 0.5075 - val_loss: 0.8805 - val_accuracy: 0.4880
Epoch 3/25
250/250 [==============================] - 22s 89ms/step - loss: 0.9210 - accuracy: 0.4890 - val_loss: 0.8764 - val_accuracy: 0.4550
Epoch 4/25
250/250 [==============================] - 22s 89ms/step - loss: 0.9016 - accuracy: 0.4930 - val_loss: 0.8704 - val_accuracy: 0.4090
Epoch 5/25
250/250 [==============================] - 22s 89ms/step - loss: 0.8983 - accuracy: 0.4865 - val_loss: 0.8384 - val_accuracy: 0.4560
Epoch 6/25
250/250 [==============================] - 23s 90ms/step - loss: 0.9099 - accuracy: 0.4865 - val_loss: 0.8651 - val_accuracy: 0.4650
Epoch 7/25
250/250 [==============================]

### Visualisasi hasil evaluasi semua model dengan hyperparameternya menggunakan Tensorboard

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

replace logs/hparam_demo/hparams_demo/0/events.out.tfevents.1550612933.goshri.c.googlers.com? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
!kill 5496